In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re


In [107]:
pd.set_option('display.max_rows', None)

# Loading Tables

In [3]:
pops = pd.read_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\Clean\\census\\pops.csv', index_col=0)

In [613]:
renda = pd.read_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\raw\\bairros\\census\\rendas_vert.csv')

In [5]:
estabs = pd.read_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\raw\\bairros\\census\\estabs_clean.csv', index_col=0)

In [6]:
emps = pd.read_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\raw\\bairros\\census\\emps_clean.csv')

In [7]:
des = pd.read_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\raw\\bairros\\census\\des_clean.csv', index_col=0)

In [8]:
sals = pd.read_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\raw\\bairros\\census\\sals_clean.csv')

## Storing All Datasets in a List

In [433]:
dfs = [pops.copy(), renda.copy(), estabs.reset_index().copy(), emps.copy(), des.copy(), sals.copy()]

# Cleaning: Uniformization of neighborhoods Names

In [434]:
for i in range(3):
    
    dfs[i].rename(columns={'Áreas de Planejamento, Regiões Administrativas e Bairros': 'Bairros'}, inplace=True)
     
    

In [435]:
dfs[4].index = dfs[1].index

In [505]:
for i in range(6): print('df'+str(i), len(dfs[i]['Bairros'].unique()))

df0 196
df1 158
df2 215
df3 216
df4 158
df5 215


In [499]:
def is_not_in(dfs):

    unis = [dfs[i]['Bairros'].unique() for i in range(6)]

    not_in = []
    for i in range(6):
        for j in range(6):
        
            if i!=j:
            
                not_in.append([i, j, unis[i][[str(o).lower().strip().replace('á','a')
                                              .replace('é','e')
                                              .replace('í','i')
                                              .replace('ó','o')
                                              .replace('ú','u') not in [str(l).lower().strip().replace('á','a')
                                                                         .replace('é','e')
                                                                         .replace('í','i')
                                                                         .replace('ó','o')
                                                                         .replace('ú','u') for l in unis[j].tolist()] for o in unis[i].tolist()]]])

    return not_in
            

In [608]:
not_in = is_not_in(dfs)

#### From the output of the cell bellow we can tell that neighborhood names from tables 0, 2, 3 and 5 follow a similar pattern againts a second pattern in 1 and 4. So we just have to concliate one group apttern with the other.

In [609]:
for i in range(30): print('From:',not_in[i][0], 'not in', not_in[i][1], len(not_in[i][2]))

From: 0 not in 1 39
From: 0 not in 2 0
From: 0 not in 3 0
From: 0 not in 4 39
From: 0 not in 5 0
From: 1 not in 0 1
From: 1 not in 2 0
From: 1 not in 3 0
From: 1 not in 4 0
From: 1 not in 5 0
From: 2 not in 0 19
From: 2 not in 1 57
From: 2 not in 3 0
From: 2 not in 4 57
From: 2 not in 5 0
From: 3 not in 0 19
From: 3 not in 1 57
From: 3 not in 2 0
From: 3 not in 4 57
From: 3 not in 5 0
From: 4 not in 0 1
From: 4 not in 1 0
From: 4 not in 2 0
From: 4 not in 3 0
From: 4 not in 5 0
From: 5 not in 0 19
From: 5 not in 1 57
From: 5 not in 2 0
From: 5 not in 3 0
From: 5 not in 4 57


## Tables 'Renda' and 'Desigualdade':

In [439]:
wrong_format = {'Acari':1, 'Parque Colúmbia':1, 'Água Santa':2, 'Encantado':2,
       'Botafogo':11, 'Urca':11, 'Camorim':15, 'Vargem Pequena':15, 'Vargem Grande':15,
       'Campinho':16, 'Vila Valqueire':16,
       'Cavalcanti':21, 'Engenheiro Leal':21, 'Vaz Lobo':21,
       'Cidade Nova':24, 'Praça da Bandeira':24, 'Cocotá':25, 'Bancários':25,
       'Deodoro':34, 'Vila Militar':34, 'Campo dos Afonsos':34, 'Jardim Sulacap':34,
       'Freguesia':40, 'Freguesia (Jacarepaguá)':41,
       'Galeão':42, 'Cidade Universitária':42,
       'Guaratiba':48, 'Barra de Guaratiba':48, 'Pedra de Guaratiba':48,
       'Jacaré':56, 'Rocha':56, 'Sampaio':56, 'Joá':63, 'Barra da Tijuca':63,
       'Mangueira':71, 'São Francisco Xavier':71, 'Maria da Graça':76, 'Del Castilho':76,
       'Moneró':78, 'Portuguesa':78, 'Recreio dos Bandeirantes':95, 'Grumari':95,
       'Ribeira':97, 'Cacuia':97, 'Santa Teresa':103, 'Cosme Velho':103,
       'São Cristóvão':105, 'Vasco da Gama':105,
       'Saúde':106, 'Gamboa':106, 'Santo Cristo':106, 'Tijuca':113, 'Alto da Boa Vista':113,
       'Vidigal':118, 'São Conrado':118, 'Vista Alegre':123, 'Irajá':123,
       'Zumbi':124, 'Pitangueiras':124, 'Praia da Bandeira':124}

In [440]:
len(np.unique([wrong_format[key] for key in wrong_format.keys()]))

27

In [441]:
right_format = {'Acari':1, 'Parque Colúmbia':106, 'Água Santa':209, 'Encantado':43,
       'Botafogo':14, 'Urca':166, 'Camorim':19, 'Vargem Pequena':172, 'Vargem Grande':171,
       'Campinho':20, 'Vila Valqueire':181,
       'Cavalcanti':26, 'Engenheiro Leal':44, 'Vaz Lobo':174,
       'Cidade Nova':28, 'Praça da Bandeira':118, 'Cocotá':31, 'Bancários':6,
       'Deodoro':42, 'Vila Militar':180, 'Campo dos Afonsos':22, 'Jardim Sulacap':82,
       'Freguesia':50, 'Freguesia (Jacarepaguá)':51,
       'Galeão':52, 'Cidade Universitária':29,
       'Guaratiba':60, 'Barra de Guaratiba':9, 'Pedra de Guaratiba':109,
       'Jacaré':77, 'Rocha':143, 'Sampaio':146, 'Joá':83, 'Barra da Tijuca':8,
       'Mangueira':91, 'São Francisco Xavier':157, 'Maria da Graça':95, 'Del Castilho':41,
       'Moneró':97, 'Portuguesa':115, 'Recreio dos Bandeirantes':122, 'Grumari':58,
       'Ribeira':140, 'Cacuia':17, 'Santa Teresa':148, 'Cosme Velho':37,
       'São Cristóvão':156, 'Vasco da Gama':173,
       'Saúde':151, 'Gamboa':53, 'Santo Cristo':149, 'Tijuca':161, 'Alto da Boa Vista':2,
       'Vidigal':176, 'São Conrado':155, 'Vista Alegre':183, 'Irajá':73,
       'Zumbi':208, 'Pitangueiras':114, 'Praia da Bandeira':116}

In [442]:
len(np.unique([right_format[key] for key in right_format.keys()]))

60

In [443]:
len(right_format.keys())

60

In [444]:
right_format_df = pd.DataFrame({'Bairros':dfs[2].sort_values(by='Bairros', ascending=True)['Bairros'].unique()})

In [ ]:

for j in [4, 1]:
    
    df0 = dfs[j].drop(dfs[j].index.tolist(), axis=0)

    for i in [0, 1]:    
    
        for key, item in wrong_format.items():    
    
            if i==1: item = item+125
    
            try: 
                x = dfs[j]['Bairros'][item]
            
                row0 = dfs[j].loc[[item]].copy()
                row0['Bairros'][item] = right_format_df.loc[right_format[key]]['Bairros']
        
                df0 = pd.concat([df0, row0], axis=0)
        
                row0.index = np.array([0])
        
            
                dfs[j].drop(item, axis=0, inplace=True)
                dfs[j] = pd.concat([dfs[j], row0], axis=0)
                    
            except:
        
                row0 = df0.loc[[item]].copy()
                row0['Bairros'][item] = right_format_df.loc[right_format[key]]['Bairros']
                row0.index = np.array([0])
        
                dfs[j] = pd.concat([dfs[j], row0], axis=0)
            

In [446]:
[len(dfs[i]['Bairros'].unique()) for i in [1, 4]]

[158, 158]

In [447]:
[len(dfs[i]['Bairros']) for i in [1, 4]]

[316, 316]

# Saving/Loading 'Rendas' and 'Desigualdade' Tables

In [452]:
#dfs[1].to_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\raw\\bairros\\census\\rendas_vert_fixed1.csv', index=False)

In [453]:
#dfs[4].to_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\raw\\bairros\\census\\des_clean_fixed1.csv', index=False)

In [667]:
dfs[1] = pd.read_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\raw\\bairros\\census\\rendas_vert_fixed1.csv')

In [453]:
dfs[4].to_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\raw\\bairros\\census\\des_clean_fixed1.csv', index=False)

# Population Table

#### The following neighborhoods names are missing from the table:

In [493]:
not_in[2][2]

array([], dtype=object)

In [494]:
not_in[10][2]

array(['Região de Planejamento 1.1', 'Vasco da Gama',
       'Região de Planejamento 2.1', 'Região de Planejamento 2.2',
       'Região de Planejamento 3.1', 'Região de Planejamento 3.2',
       'Região de Planejamento 3.3', 'Região de Planejamento 3.4',
       'Região de Planejamento 3.5', 'XXXI Vigário Geral',
       'Região de Planejamento 3.6', 'Região de Planejamento 3.7',
       'Região de Planejamento 4.1', 'Região de Planejamento 4.2',
       'Região de Planejamento 5.1', 'Gericinó',
       'Região de Planejamento 5.2', 'Região de Planejamento 5.3',
       'Região de Planejamento 5.4'], dtype=object)

#### Note: neighborhoods 'Vasco da Gama', 'XXXI Vigário Geral' and 'Gericinó' are currently missing from this table, so they are going to be excluded from all posterior analysis.

#### That's alright, we are going to save those and those other ones for later.

In [ ]:
freguesia ilha: 114/50
freguesia jaca: 151/51
vila kosmos: 92/178

In [491]:
dfs[0].iloc[114,0] = right_format_df['Bairros'][50]
dfs[0].iloc[151,0] = right_format_df['Bairros'][51]
dfs[0].iloc[92,0] = right_format_df['Bairros'][178]

# Testing: 

In [570]:
not_in = is_not_in(dfs)

In [503]:
not_in[2][2]

array([], dtype=object)

# Missing Values

In [561]:
sum([True if (type(j)==float) else False for j in dfs[3]['Bairros']])

56

In [562]:
len(dfs[3].dropna(subset=['Bairros']))

3010

In [563]:
len(dfs[3])

3066

In [565]:
dfs[3] = dfs[3].dropna(subset=['Bairros'])

# Final Formatting

In [672]:
for i in range(6): 
    dfs[i]['Bairros'].replace(np.nan, '')
    dfs[i]['Bairros'] = [j.lower().strip().replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u') for j in dfs[i]['Bairros']]

# Saving Loading Population Table:

In [602]:
#dfs[0].to_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\Clean\\census\\pops_clean_fixed.csv')

In [666]:
dfs[0] = pd.read_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\Clean\\census\\pops_clean_fixed.csv', index_col=0)

# Saving the Others:

In [606]:
#dfs[2].to_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\raw\\bairros\\census\\estabs_clean_fixed.csv', index=False)

In [605]:
#dfs[3].to_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\raw\\bairros\\census\\emps_clean_fixed.csv', index=False)

In [607]:
#dfs[5].to_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\raw\\bairros\\census\\sals_clean_fixed.csv', index=False)

# Combining All Tables:

## Reducing Time Series To the Mean.

In [634]:
dfs[1].reset_index(inplace=True, drop=True)

In [635]:
for i in range(6): print(len(dfs[i]['Bairros']))

196
316
3010
3010
316
3010


# 

In [673]:
dfs_grp = []

dfs_grp.append(dfs[1].groupby('Bairros', as_index=False).mean().copy())

print(len(dfs_grp[0]))

dfs_grp.append(dfs[2].groupby('Bairros', as_index=False).mean().copy())

print(len(dfs_grp[1]))

dfs_grp.append(dfs[3].groupby('Bairros', as_index=False).mean().copy())

print(len(dfs_grp[2]))

dfs_grp.append(dfs[4].groupby('Bairros', as_index=False).mean().copy())

print(len(dfs_grp[3]))

dfs_grp.append(dfs[5].groupby('Bairros', as_index=False).mean().copy())

print(len(dfs_grp[4]))

158
215
215
158
215


## Merging

In [717]:
hood = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(dfs[0], dfs_grp[0], on='Bairros'),
                                                            dfs_grp[3], on='Bairros'),
                                                            dfs_grp[1], on='Bairros'),
                                                            dfs_grp[2], on='Bairros'), 
                                                            dfs_grp[4], on='Bairros')

# Saving Complete Merged Table

In [719]:
#hood.to_csv('\\Users\luisr\Desktop\Repository\IBM_courses\Course Work\Assignments\Final Capstone Project\\rio dataframes\\Outside Data\\Clean\\census\\rio_complete.csv', index=False)

In [707]:
corr = hood.iloc[:, 1:].astype(float).corr().copy()

In [ ]:
fig = plt.figure(figsize=(40, 40))
ax0 = fig.add_subplot(111)

sns.heatmap(corr.iloc[:, :60], linewidths=0.5, cmap='RdBu', ax=ax0)